In [ ]:
import pandas as pd
from IPython.display import display
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, Sequential, Input, Model
from keras.layers import *
import pickle

In [ ]:
state_names = ["Alabama", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Idaho", "Illinois", "Indiana", 
               "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", 
               "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", 
               "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", 
               "West Virginia", "Wisconsin", "Wyoming"]

In [ ]:
us_mean_temp = []
us_minmax_temp = []
us_precip = []

In [ ]:
path = ""
for i in state_names:
    mean_temp = pd.read_csv(path + i.lower().replace(" ","") + "_mean_temp.csv")
    minmax_temp = pd.read_csv(path + i.lower().replace(" ","") + "_minmax_temp.csv")
    precip = pd.read_csv(path + i.lower().replace(" ","") + "_precip.csv")
    
    us_mean_temp.append(mean_temp)
    us_minmax_temp.append(minmax_temp)
    us_precip.append(precip)

In [ ]:
increasing_precip = ["Alabama", "Arizona", "Connecticut", "Delaware", "Florida", "Georgia", "Illinois",
                    "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maryland", "Massachusetts",
                    "Mississippi", "Missouri", "New Hampshire", "New Jersey", "New York", "North Carolina",
                    "North Dakota", "Ohio", "Oklahoma", "Pennsylvania", "South Dakota", "Tennessee", "Texas",
                    "Virginia", "West Virginia"]

In [ ]:
precip_pos = [state_names.index(i) for i in increasing_precip]

In [ ]:
precip_neg = [i for i in range(len(state_names)) if not i in precip_pos]

In [ ]:
names_tx = ["East Texas","Edwards Plateau","High Plains","Low Rolling Plains","Lower Valley",
               "North Central","South Central","South Texas","Trans Pecos","Upper Coast"]

In [ ]:
tx_mean_temp = []
tx_minmax_temp = []
tx_precip = []

In [ ]:
path = ""
for i in names_tx:
    mean_temp = pd.read_csv(path + i.lower().replace(" ","") + "_mean_temp.csv")
    minmax_temp = pd.read_csv(path + i.lower().replace(" ","") + "_minmax_temp.csv")
    precip = pd.read_csv(path + i.lower().replace(" ","") + "_precip.csv")
    
    tx_mean_temp.append(mean_temp)
    tx_minmax_temp.append(minmax_temp)
    tx_precip.append(precip)

In [ ]:
def get_adjustments(data_list_minmax, data_list_mean, start, end):
    adjustments = np.zeros((len(data_list_mean),2))
    for ii in range(len(data_list_mean)):
        lowest_error_min = np.inf
        lowest_error_max = np.inf
        best_adjustment_min = 0
        best_adjustment_max = 0
        for i in np.arange(start, end, 0.01):
            error_min = np.mean(np.abs(data_list_minmax[ii].iloc[:,-1] + i - data_list_mean[ii].iloc[:,-1]))
            error_max = np.mean(np.abs(data_list_minmax[ii].iloc[:,-2] - i - data_list_mean[ii].iloc[:,-1]))
            if error_min < lowest_error_min:
                lowest_error_min = error_min
                best_adjustment_min = i
            if error_max < lowest_error_max:
                lowest_error_max = error_max
                best_adjustment_max = i
        adjustments[ii,0] = best_adjustment_max
        adjustments[ii,1] = best_adjustment_min
    return adjustments

In [ ]:
adjustments_tx = get_adjustments(tx_minmax_temp, tx_mean_temp, 5, 15)

In [ ]:
adjustments_us = get_adjustments(us_minmax_temp, us_mean_temp, 0, 20)

In [ ]:
colors=["tab:blue","tab:orange","tab:green","tab:red","tab:purple","tab:brown","tab:pink","tab:gray",
       "tab:olive","tab:cyan","k","m","yellow","lightcoral","darkblue","lightgreen","burlywood"]

In [ ]:
def plot_trend(data_list, ewm, title, names):
    plt.figure(figsize=(15,10))
    for i in range(len(data_list)):
        plt.plot(data_list[i].iloc[:,-1].ewm(ewm).mean().values, label=names[i], c=colors[i])
        # plt.plot(data_list[i].iloc[:,-1].ewm(ewm).mean().values / 90, label=names[i], c=colors[i]) # uncomment for precipitation
    plt.xticks(ticks=np.arange(1,46,5), labels=np.arange(1980,2025,5), fontsize=12)
    plt.yticks(fontsize=12)
    plt.ylabel("Temperature (°F.)", fontsize=12)
#     plt.ylabel("Precipitation (in.)", fontsize=12) # uncomment for precipitation
    plt.title(title + "(1979-2021)", fontsize=20)
    plt.legend(bbox_to_anchor=(1.03,1), fontsize=12)
    plt.show()

In [ ]:
plot_trend(tx_mean_temp, 5, "Texas Climate Divisions Mean Summer Temperature", names_tx)

In [ ]:
def f(x):
    return (1/100)*x - np.exp(-0.01*x)*np.sin(0.6*x)*0.5*x**0.3 + 81.5

In [ ]:
fig, axs = plt.subplots(2,5,figsize=(20,7))
coords = [(i,j) for j in range(5) for i in range(2)]
x = np.linspace(0,43,100)
for i in range(7,len(south_fl)):
    if i == 7:
        axs[coords[i-7]].plot(south_fl[i].iloc[:,-1].ewm(0).mean().values)
        axs[coords[i-7]].plot(x, f(x), label="T(t)")
        axs[coords[i-7]].set_title(names[i],fontsize=14)
        axs[coords[i-7]].legend(fontsize=14)
    else:
        axs[coords[i-7]].plot(south_fl[i].iloc[:,-1].ewm(0).mean().values)
        axs[coords[i-7]].plot(x, f(x))
        axs[coords[i-7]].set_title(names[i],fontsize=14)
plt.show()

In [ ]:
lookback_len = 20 # how far to look back
pred_len = 10 # how many timesteps to predict

In [ ]:
# we need continuous sequences of length lookback_len and pred_len to serve as X and Y respectively 
def create_examples(lookback_len, pred_len, data):
    X = []
    Y = []
    for i in range(0, data.shape[0] - lookback_len - pred_len):
        X.append(data[i:i+lookback_len].reshape(-1,1))
        Y.append(data[i+lookback_len:i+lookback_len+pred_len].reshape(-1,1))
    X = np.array(X)
    Y = np.array(Y)
    return X,Y

In [ ]:
def get_data(ewm_len,data_list,lookback_len=20,pred_len=10):
    X = [] 
    Y = []
    for i in range(len(data_list)): # assembling X and Y using data from all counties
        data = data_list[i].iloc[:,-1].ewm(ewm_len).mean().dropna().values
        x, y = create_examples(lookback_len, pred_len, data)
        X.append(x)
        Y.append(y)
    X = np.array(X)
    X = X.reshape(X.shape[0]*X.shape[1],lookback_len,1)
    
    Y = np.array(Y)
    Y = Y.reshape(Y.shape[0]*Y.shape[1],pred_len,1)
    return X, Y

In [ ]:
variable = [us_precip[i] for i in precip_neg]
ewm_len = 5
X, Y = get_data(ewm_len,variable,lookback_len=lookback_len,pred_len=pred_len)

In [ ]:
def build_LSTM(input_shape, num_units, dropout_rate, pred_len, initializer):
    input_layer = Input(shape=input_shape)
    lstm1 = Dropout(dropout_rate)(input_layer)
    lstm1 = LSTM(num_units, kernel_initializer=initializer)(lstm1)
    lstm1 = Activation("tanh")(lstm1)
    output = Dense(pred_len)(lstm1)
    model = Model(inputs=[input_layer], outputs=[output])
    return model 

In [ ]:
# In each state/climate division, there are 43 data examples corresponding to 1979-2021,
# meaning there are 43-(20+10)=13 time series sequences for each county.

# We need to reserve the last few examples from each county as test data.
# A random split cannot be used, as that would allow the model to see future data while training.

In [ ]:
train_idxs = []
test_idxs = []

for i in range(7,261,9): # for USA precip pos
# for i in range(7,171,9): # for USA precip neg
# for i in range(9,633,13): # for USA temp
# for i in range(9,139,13): # for Texas
    test_idxs += [i,i+1]
for i in range(X.shape[0]):
    if not i in test_idxs:
        train_idxs.append(i)

In [ ]:
len(train_idxs) + len(test_idxs) == X.shape[0]

In [ ]:
X_t = X[train_idxs]
X_v = X[test_idxs]

y_t = Y[train_idxs]
y_v = Y[test_idxs]

In [ ]:
X_t_means = np.mean(X_t, axis=1).reshape(-1,1,1)
X_t_stds = np.std(X_t, axis=1).reshape(-1,1,1)
X_t = (X_t - X_t_means) / X_t_stds
y_t = (y_t - X_t_means) / X_t_stds

X_v_means = np.mean(X_v, axis=1).reshape(-1,1,1)
X_v_stds = np.std(X_v, axis=1).reshape(-1,1,1)
X_v = (X_v - X_v_means) / X_v_stds
y_v = (y_v - X_v_means) / X_v_stds

In [ ]:
initializer = tf.keras.initializers.GlorotNormal(seed=0)

num_units = 32
dropout_rate = 0.2

model = build_LSTM((X_t.shape[1:]), num_units, dropout_rate, pred_len, initializer)
model.compile(optimizer="adam", loss="mae", metrics=["mse"])
history_nn = model.fit(X, Y, batch_size=8, epochs=75)
history_nn = model.fit(X_t, y_t, validation_data=(X_v, y_v), batch_size=8, epochs=150)

In [ ]:
plt.plot(pd.DataFrame(history_nn.history).loss)
plt.plot(pd.DataFrame(history_nn.history).val_loss)
plt.show()

In [ ]:
path = ""
# model.save_weights(path)
# model.load_weights(path)

In [ ]:
def forecast(data_list, names_list, num_rows, num_cols, idx_legend):
    fig, axs = plt.subplots(num_rows,num_cols,figsize=(20,15))
    coords = [(i,j) for j in range(num_cols) for i in range(num_rows)]
    for county in range(len(data_list)):
#     for county in precip_neg:
        all_x = data_list[county].iloc[:,-1].ewm(0).mean().dropna().values
        x = all_x[-lookback_len:]
        means = np.mean(x)
        stds = np.std(x)
        x = (x - means) / stds
        y = []   
        for i in range(3): # 40 years into the future
            pred = model.predict(x.reshape(1,lookback_len,1))
            pred = pred * stds + means
            x = x * stds + means
            y.append(pred[0].flatten())
            x = np.append(x.reshape(-1)[-10:], pred.reshape(-1))
            means = np.mean(x)
            stds = np.std(x)
            x = (x - means) / stds
#         y = np.array(y) / 90 # Uncomment for precipitation (3 months x 30 days per month)
#         x = x / 90
        axs[coords[county]].plot(np.arange(0,43), all_x, label="Past")
#         axs[coords[county]].plot(np.arange(4,43), all_x, label="Past") when rolling=5 is used
        axs[coords[county]].axvspan(42,72, facecolor='0.2', alpha=0.2)
        axs[coords[county]].plot(np.arange(42,44), np.append(all_x[-1], np.array(y).flatten()[0]), c="tab:orange")
        axs[coords[county]].plot(np.arange(43,73), np.array(y).flatten().reshape(-1), label="Future")
        axs[coords[county]].set_title(names_list[county], fontsize=14)
        axs[coords[county]].set_xticks(ticks=np.arange(2,82,10), labels=np.arange(1980,2060,10))
        if county == idx_legend: 
            axs[coords[county]].legend(bbox_to_anchor=(1.03,1), fontsize=14)
    plt.subplots_adjust(hspace=0.4)
    plt.show()

In [ ]:
forecast(tx_precip, names_tx, 5, 2, 5)

In [ ]:
def forecast_df(data_list, names):
    future = np.zeros((len(data_list),3*pred_len))
    for county in range(len(data_list)):
        all_x = data_list[county].iloc[:,-1].ewm(0).mean().dropna().values
        x = all_x[-lookback_len:]
        means = np.mean(x)
        stds = np.std(x)
        x = (x - means) / stds
        y = []   
        for i in range(3): # 30 years into the future
            pred = model.predict(x.reshape(1,lookback_len,1))
            pred = pred * stds + means
            future[county,i*pred_len:(i+1)*pred_len] = pred.reshape(-1
            x = x * stds + means
            y.append(pred[0].flatten())
            x = np.append(x.reshape(-1)[-10:], pred.reshape(-1))
            means = np.mean(x)
            stds = np.std(x)
            x = (x - means) / stds
    future = pd.DataFrame(future)
    return future

In [ ]:
us_future_mean_temp = forecast_df(us_mean_temp, state_names)

In [ ]:
us_future_precip = forecast_df(us_precip, state_names)

In [ ]:
for i in range(us_future_precip.shape[0]):
    us_future_precip.iloc[i] = us_future_precip.iloc[i].ewm(5).mean()

In [ ]:
us_future_precip_decades = np.zeros((48,3))
for state in precip_pos:
    lst = []
    for i in range(8, 38, 10):
        lst.append(us_future_precip.iloc[state,i])
    us_future_precip_decades[state] = lst  
us_future_precip_decades = pd.DataFrame(us_future_precip_decades)
us_future_precip_decades["State"] = state_names
us_future_precip_decades.columns = [2030,2040,2050,"State"]

In [ ]:
# Go back and generate predictions for states with decreasing precipitations

In [ ]:
temporary = forecast_df(us_precip, state_names)

In [ ]:
for i in range(temporary.shape[0]):
    temporary.iloc[i] = temporary.iloc[i].ewm(5).mean()

In [ ]:
for state in precip_neg:
    lst = []
    for i in range(8, 38, 10):
        lst.append(temporary.iloc[state,i])
    us_future_precip_decades.iloc[state,:-1] = lst  

In [ ]:
us_future_precip_decades.iloc[:,:-1] /= 90

In [ ]:
us_future_mean_temp_decades = np.zeros((48,3))
for state in range(len(us_mean_temp)):
    lst = []
    for i in range(8, 38, 10):
        lst.append(us_future_mean_temp.iloc[state,i])
    us_future_mean_temp_decades[state] = lst  
us_future_mean_temp_decades = pd.DataFrame(us_future_mean_temp_decades)
us_future_mean_temp_decades["State"] = state_names
us_future_mean_temp_decades.columns = [2030,2040,2050,"States"]

In [ ]:
us_future_min_temp = us_future_mean_temp_decades.copy()
us_future_max_temp = us_future_mean_temp_decades.copy()

for i in range(len(us_mean_temp)):
    us_future_min_temp.iloc[i,:-1] -= adjustments_us[i,1]
    us_future_max_temp.iloc[i,:-1] += adjustments_us[i,0]

In [ ]:
path = ""
us_future_mean_temp_decades.to_csv(path)
us_future_min_temp.to_csv(path)
us_future_max_temp.to_csv(path)
us_future_precip_decades.to_csv(path)

In [ ]:
tx_future_mean_temp = forecast_df(tx_mean_temp, names_tx)

In [ ]:
for i in range(tx_future_mean_temp.shape[0]):
    tx_future_mean_temp.iloc[i] = tx_future_mean_temp.iloc[i].ewm(10).mean()

In [ ]:
tx_future_precip = forecast_df(tx_precip, names_tx)
tx_future_precip /= 90

In [ ]:
for i in range(tx_future_precip.shape[0]):
    tx_future_precip.iloc[i] = tx_future_precip.iloc[i].ewm(10).mean()

In [ ]:
tx_future_mean_temp_decades = np.zeros((10,4))
for county in range(len(tx_mean_temp)):
    lst = []
    for i in range(8, 48, 10):
        lst.append(tx_future_mean_temp.iloc[county,i])
    tx_future_mean_temp_decades[county] = lst  
tx_future_mean_temp_decades = pd.DataFrame(tx_future_mean_temp_decades)
tx_future_mean_temp_decades["Division"] = names_tx
tx_future_mean_temp_decades.columns = [2030,2040,2050,2060,"Division"]

In [ ]:
tx_future_precip_decade = np.zeros((10,4))
for county in range(len(tx_precip)):
    lst = []
    for i in range(8, 48, 10):
        lst.append(tx_future_precip.iloc[county,i])
    tx_future_precip_decade[county] = lst  
tx_future_precip_decade = pd.DataFrame(tx_future_precip_decade)
tx_future_precip_decade["Division"] = names_tx
tx_future_precip_decade.columns = [2030,2040,2050,2060,"Division"]

In [ ]:
tx_future_min_temp = tx_future_mean_temp_decades.copy()
tx_future_max_temp = tx_future_mean_temp_decades.copy()

for i in range(len(tx_mean_temp)):
    tx_future_min_temp.iloc[i,:-1] -= adjustments_tx[i,1]
    tx_future_max_temp.iloc[i,:-1] += adjustments_tx[i,0]

In [ ]:
tx_future_min_temp = tx_future_mean_temp.copy()
tx_future_max_temp = tx_future_mean_temp.copy()

for i in range(len(tx_mean_temp)):
    tx_min_temp.iloc[i,:-1] -= adjustments_tx[i,1]
    tx_max_temp.iloc[i,:-1] += adjustments_tx[i,0]

In [ ]:
path = ""
tx_future_mean_temp_decades.to_csv(path)
tx_future_min_temp.to_csv(path)
tx_future_max_temp.to_csv(path)
tx_future_precip_decades.to_csv(path)